In [37]:
import configparser
import psycopg2
import boto3
import json
import pandas as pd
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [38]:
# define configuration parameter
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [39]:
KEY=config.get('AWS','key')
SECRET= config.get('AWS','secret')
ARN = config.get('IAM_ROLE', 'ARN')
LOG_DATA= config.get('S3', 'LOG_DATA')
LOG_JSONPATH= config.get('S3', 'LOG_JSONPATH')
SONG_DATA= config.get('S3', 'SONG_DATA')

#### Load DWH params from a file

In [40]:
import pandas as pd
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


#### Create Clients for EC2, S3, IAM, and Redshift

In [41]:
import boto3

ec2 =  boto3.resource('ec2',
                                       region_name="us-west-2",
                                       aws_access_key_id = KEY,
                                       aws_secret_access_key=SECRET
                                       )

s3 =  boto3.resource('s3',
                                       region_name="us-west-2",
                                       aws_access_key_id = KEY,
                                       aws_secret_access_key=SECRET
                                       )

iam =  boto3.client('iam',
                                       region_name="us-west-2",
                                       aws_access_key_id = KEY,
                                       aws_secret_access_key=SECRET
                                       )

redshift = boto3.client('redshift',
                                       region_name="us-west-2",
                                       aws_access_key_id = KEY,
                                       aws_secret_access_key=SECRET
                                       )
                                        

### Add redshift database and IAM role info to dwh.cfg.

In [42]:
import json
# Creating a new IAM Role    
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
        Path = '/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument = json.dumps(
            {'Statement':[{'Action':'sts:AssumeRole',
                          'Effect':'Allow',
                          'Principal':{'Service':'redshift.amazonaws.com'}}],
            'Version':'2012-10-17'})
    )
    
except Exception as e:
    print(e)

1.1 Creating a new IAM Role


In [43]:
# Add policy to IAM role
print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                                          PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                                          )['ResponseMetadata']['HTTPStatusCode']

1.2 Attaching Policy


200

In [44]:
# Confirm IAM and ARN
print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::972027451368:role/dwhRole


In [45]:
# Create cluster in redshift
try:
    response = redshift.create_cluster(  
        # TODO: add parameters for hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        # TODO: add parameters for identifiers & credentials
        DBName = DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        # TODO: add parameter for role (to allow s3 access)
         IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists


In [46]:
# Confirm status of the redshift
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.c5c5qnu0k1ew.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-803e86f8
7,NumberOfNodes,4


In [47]:
# Get DWH ENDPOINT and ARN
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN )

DWH_ENDPOINT ::  dwhcluster.c5c5qnu0k1ew.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::972027451368:role/dwhRole


In [48]:
# Open TCP Port to access the cluster endpoint
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name,  # TODO: fill out
        CidrIp='0.0.0.0/0',  # TODO: fill out
        IpProtocol='TCP',  # TODO: fill out
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-3d742973')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


#### Delete Command

In [49]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.c5c5qnu0k1ew.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2019, 11, 20, 15, 6, 41, 929000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-3d742973',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-803e86f8',
  'AvailabilityZone': 'us-west-2a',
  'PreferredMaintenanceWindow': 'wed:11:30-wed:12:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': False,
  'Ia

In [50]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!

{'ResponseMetadata': {'RequestId': '198a0413-6c83-4261-9d5c-c3d2c58e8331',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '198a0413-6c83-4261-9d5c-c3d2c58e8331',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Wed, 20 Nov 2019 15:55:15 GMT'},
  'RetryAttempts': 0}}